要模拟一个情景，其中有600个人站成一排，每次随机杀掉一个奇数位置的人，目的是找出哪个位置的人最安全。具体实现可以分为以下几个步骤和解释：

### 1. 问题理解

- **初始状态**：有600个人站成一排，编号从1到600。
- **杀人规则**：每一轮中，随机选择当前存活的奇数编号的人，杀掉他们。
- **最终目标**：运行多次模拟，统计每个人被杀的轮次，找出哪些编号的人最晚被杀，或者可能存活到最后。

### 2. 实现思路

实现的关键在于通过多次模拟，记录每个编号的人在每次模拟中被杀的轮次，然后通过统计找到最安全的位置。

### 3. 关键步骤解释

1. **`random_kill(n)` 和 `random_kill_list(alive_ids)` 函数**：
   - `random_kill(n)`：在给定范围内随机选择一个奇数位置的人。
   - `random_kill_list(alive_ids)`：从当前存活的人中，选择一个奇数位置的人进行淘汰。

2. **`run_simulation_numpy(num_people)` 函数**：
   - `ids`：表示所有人的编号，从1到600。
   - `status`：表示每个人的生存状态，1表示存活，0表示被淘汰。
   - `killed_turn`：记录每个人被淘汰的回合，初始值为-1表示尚未被淘汰。
   - 每轮循环中，`alive_ids`会筛选出当前存活的编号，然后随机淘汰其中一个奇数编号的人。
   - 淘汰后更新 `status` 和 `killed_turn`，最后将结果存储在一个DataFrame中。

3. **多次模拟**：
   - 通过循环运行 `run_simulation_numpy(num_people)` N次（例如20000次），在每次模拟中记录每个人的淘汰回合。
   - 结果存储在 `df_simul` DataFrame中，每一列表示一次模拟的结果。



In [3]:
import pandas as pd
import numpy as np
import random
import warnings
import seaborn as sns
from pylab import mpl, plt

# best font and style settings for notebook
warnings.filterwarnings('ignore')
sns.set_style("white")
mpl.rcParams['font.family'] = '微软雅黑'
warnings.filterwarnings('ignore')

from tqdm import tqdm_notebook


def random_kill(n):
    # 生成一个随机的奇数，范围从0到n-1
    return random.choice([i for i in range(n) if i % 2 == 0])


def random_kill_list(alive_ids):
    n = len(alive_ids)
    return alive_ids[random_kill(n)]


def run_simulation_numpy(num_people):

    # 初始化 NumPy 数组
    ids = np.arange(1, num_people + 1)
    status = np.ones(num_people, dtype=int)  # 1表示存活
    killed_turn = np.full(num_people, -1)  # 初始化淘汰回合

    for turn in range(num_people):
        alive_ids = ids[status == 1]  # 获取当前存活的ID
        if len(alive_ids) == 0:  # 如果没有存活者，提前退出循环
            break

        kill_id = random_kill_list(alive_ids) - 1  # 选择要淘汰的ID

        status[kill_id] = 0  # 标记为淘汰
        killed_turn[kill_id] = turn + 1  # 记录淘汰回合

    # 转换为 DataFrame
    df = pd.DataFrame({
        "id": ids,
        "status": status,
        "killed_turn": killed_turn
    })

    return df


#
num_people = 600
N = 50000
df_simul = pd.DataFrame({"id": range(1, 1 + num_people)})
for i in tqdm_notebook(range(N)):
    result_df = run_simulation_numpy(num_people)
    df_simul[f'turn_{i}'] = result_df['killed_turn']

result_df

  0%|          | 0/50000 [00:00<?, ?it/s]

,id,status,killed_turn
0,1,0,347
1,2,0,412
2,3,0,415
3,4,0,521
4,5,0,214
...,...,...,...
595,596,0,126
596,597,0,488
597,598,0,344
598,599,0,349


In [4]:
df_simul.iloc[df_simul.iloc[:,1:].idxmax()]

,id,turn_0,turn_1,turn_2,turn_3,turn_4,turn_5,turn_6,turn_7,turn_8,...,turn_49990,turn_49991,turn_49992,turn_49993,turn_49994,turn_49995,turn_49996,turn_49997,turn_49998,turn_49999
380,381,600,599,332,167,385,520,44,193,108,...,347,233,369,386,349,547,352,259,479,103
420,421,105,600,546,34,20,584,229,372,300,...,107,484,213,186,457,193,168,242,166,434
402,403,474,128,600,197,357,179,552,322,41,...,141,275,162,322,14,344,71,278,404,479
502,503,90,277,532,600,254,526,287,147,368,...,104,66,38,140,156,444,11,6,95,148
578,579,248,386,433,451,600,360,51,336,375,...,458,481,546,534,561,427,95,463,250,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,593,188,134,35,318,169,435,159,94,69,...,322,551,351,390,281,600,340,134,154,487
82,83,426,262,119,162,467,307,459,74,453,...,317,498,467,48,441,35,600,78,61,9
530,531,324,37,98,113,190,382,193,466,477,...,134,375,253,506,243,417,580,600,55,43
144,145,139,142,28,277,108,226,335,285,11,...,199,567,199,44,231,162,140,196,600,215


: 

In [5]:
# 统计每个ID成为最后一个被淘汰的次数
last_killed = df_simul.iloc[:, 1:].idxmax()
last_killed_counts = {}
for i in range(1, len(df_simul )+1):
    last_killed_counts[i] = 0
# print(last_killed_counts.keys())
for i in df_simul.loc[df_simul.iloc[:, 1:].idxmax()]['id']:
    last_killed_counts[i] += 1
last_killed_counts = pd.DataFrame({
    'id': last_killed_counts.keys(),
    'win_count': last_killed_counts.values()
})

last_killed_counts.plot(x='id',y='win_count',figsize=(12,5))

In [ ]:
last_killed_counts.sort_values('win_count',ascending=False)

,id,win_count
599,600,80
598,599,79
591,592,78
576,577,77
586,587,77
...,...,...
8,9,0
9,10,0
11,12,0
1,2,0


In [ ]:
df_simul['average_alive_turns'] = df_simul.T.mean()
df_simul[['id', 'average_alive_turns']].sort_values(by='average_alive_turns',
                                                    ascending=False)

,id,average_alive_turns
1,2,333.194140
3,4,321.787661
5,6,318.514724
7,8,315.861057
11,12,312.203490
...,...,...
8,9,284.584771
6,7,281.164492
4,5,269.539373
2,3,253.997750


In [ ]:
df_simul[['id', 'average_alive_turns']].plot(x='id',y='average_alive_turns',figsize=(13,5))

NameError: name 'df_simul' is not defined

In [ ]:
!jupyter nbconvert --to html random_kill_final.ipynb

[NbConvertApp] Converting notebook random_kill_final.ipynb to html
[NbConvertApp] Writing 426751 bytes to random_kill_final.html
